In [1]:
# Create a function to fetch the stock data based on the ticker
def get_stock_data(ticker, years=1):
    start_date=(datetime.datetime.today() - datetime.timedelta(days=365*years)).strftime('%Y-%m-%d')
    end_date=datetime.datetime.today().strftime('%Y-%m-%d')
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

In [2]:
import dash 
from dash import dcc, html, Input, Output, dash_table 
import dash_bootstrap_components as dbc
import datetime as datetime
from dash.dependencies import Input, Output, State
import yfinance as yf
import requests  # To fetch data from an external API
import os

# Replace 'YOUR_API_KEY' with your actual NewsAPI key
news_api_key = os.environ.get('NEWS_API_KEY')

start_date = (datetime.datetime.today() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')
end_date = datetime.datetime.today().strftime('%Y-%m-%d')
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Stock Performance Dashboard"),
        ]),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Input(
                id='stock-input',
                type='text',
                placeholder='Enter a stock symbol...',
                debounce=True,
                style={'width': '100%'}
            ),
        ], width=4),
        dbc.Col([
            dcc.Dropdown(
                id='stock-dropdown',
                multi=True,
                placeholder='Selected stocks...',
                style={'width': '100%'}
            ),
        ], width=4),
        dbc.Col([
            dcc.DatePickerRange(
                id='date-picker',
                start_date=start_date,
                end_date=end_date,
                display_format='YYYY-MM-DD'
            ),
        ], width=4),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='stock-graph')
        ]),
    ]),
], fluid=True)

def fetch_stock_news(symbol, start_date, end_date):
    """
    Fetches significant news articles for a given stock symbol using NewsAPI.
    """
    url = f"https://newsapi.org/v2/everything?q={symbol}&from={start_date}&to={end_date}&sortBy=publishedAt&apiKey={news_api_key}"
    response = requests.get(url)
    articles = response.json().get('articles', [])

    # Process articles to create events with date, title, and description
    events = []
    for article in articles:
        # Ensure the article has a published date
        if 'publishedAt' in article:
            event_date = article['publishedAt'][:10]  # Extract date in 'YYYY-MM-DD' format
            events.append({
                'date': event_date,
                'event': article['title'],  # Use the article title as the event description
                'description': article['description']  # Use article description if needed
            })

    return events

@app.callback(
    Output('stock-dropdown', 'options'),
    Output('stock-dropdown', 'value'),
    Input('stock-input', 'value'),
    State('stock-dropdown', 'value'),
    State('stock-dropdown', 'options')
)
def update_dropdown(search_input, selected_symbols, existing_options):
    if not search_input:
        raise dash.exceptions.PreventUpdate

    # Initialize options if None
    if existing_options is None:
        existing_options = []

    # Fetch stock data from yfinance (or another API)
    try:
        stock_info = yf.Ticker(search_input).info
        stock_name = stock_info.get('longName', 'Unknown')
        option = {'label': f"{search_input.upper()} - {stock_name}", 'value': search_input.upper()}
        
        # Add new stock to options if not already present
        if option not in existing_options:
            existing_options.append(option)

        # Update selected symbols
        if selected_symbols is None:
            selected_symbols = []
        
        if search_input.upper() not in selected_symbols:
            selected_symbols.append(search_input.upper())
    except Exception as e:
        print(f"Error fetching stock info: {e}")
    
    return existing_options, selected_symbols

@app.callback(
    Output('stock-graph', 'figure'),
    Input('stock-dropdown', 'value'),
    Input('date-picker', 'start_date'),
    Input('date-picker', 'end_date')
)
def update_graph(selected_symbols, start_date, end_date):
    if not selected_symbols:
        return {'data': [], 'layout': {'title': 'Stock Closing Prices'}}

    data = []
    annotations = []  # List to hold annotations
    for symbol in selected_symbols:
        try:
            df = yf.download(symbol, start=start_date, end=end_date)
            if not df.empty:
                data.append({
                    'x': df.index, 
                    'y': df['Close'], 
                    'type': 'line', 
                    'name': symbol,
                    'hovertemplate': f"<b>{symbol}</b><br>Date: {{%{{x|%Y-%m-%d}}}}<br>Close: {{%{{y:.2f}}}}<extra></extra>"
                })

                # Fetch news dynamically using NewsAPI
                events = fetch_stock_news(symbol, start_date, end_date)
                for event in events:
                    event_date = event['date']
                    if event_date in df.index:  # Ensure the event date is within the fetched data
                        annotations.append({
                            'x': event_date, 
                            'y': df.loc[event_date]['Close'],
                            'xref': 'x', 
                            'yref': 'y',
                            'text': event['event'],
                            'showarrow': True, 
                            'arrowhead': 2, 
                            'ax': -40, 
                            'ay': -30
                        })
            else:
                print(f"No data found for {symbol} in the specified date range.")
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

    # Update the layout with hovermode and annotations
    layout = {
        'title': 'Stock Closing Prices',
        'hovermode': 'x unified',
        'annotations': annotations  # Include annotations in the layout
    }

    return {'data': data, 'layout': layout}

if __name__ == '__main__':
    app.run_server(debug=True)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [13]:
def fetch_stock_news(symbol, start_date, end_date):
    """
    Fetches significant news articles for a given stock symbol using NewsAPI.
    """
    url = f"https://newsapi.org/v2/everything?q={symbol}&from={start_date}&to={end_date}&sortBy=publishedAt&apiKey={news_api_key}"
    response = requests.get(url)
    print(f"NewsAPI Response Status: {response.status_code}")
    if response.status_code != 200:
        print("Error fetching news:", response.json())
        return []

    articles = response.json().get('articles', [])

    # Process articles to create events with date, title, and description
    events = []
    for article in articles:
        # Ensure the article has a published date
        if 'publishedAt' in article:
            event_date = article['publishedAt'][:10]  # Extract date in 'YYYY-MM-DD' format
            events.append({
                'date': event_date,
                'event': article['title'],  # Use the article title as the event description
                'description': article['description']  # Use article description if needed
            })

    print(f"Fetched {len(events)} events for symbol {symbol}")
    return events

In [3]:
print(start_date)
print(end_date)

2023-09-06
2024-09-05


In [15]:
news = fetch_stock_news('AAPL', start_date, end_date)
news

NewsAPI Response Status: 429
Error fetching news: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}


[]